In [22]:
from PIL import Image
import numpy as np
import os
import pandas as pd

import glob
import argparse
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import utils

from __future__ import print_function, division   # Ensures Python3 printing & division standard
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.pylab import rcParams

import numpy as np
import pandas as pd 
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score,mean_squared_error, mean_absolute_error, r2_score, roc_curve,auc
from sklearn.tree import DecisionTreeClassifier, plot_tree,export_graphviz
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier

from scipy.stats import randint, poisson
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb
from xgboost import plot_tree

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Sequential, Model


from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta


from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from ipywidgets import interactive
from graphviz import Source
from IPython.display import SVG

import csv

%matplotlib qt 

In [4]:
# Load the data
metadata_file = "metadata19_hmineq0.0_tmin20050000_mean_grid_20.csv"
glathida_rgis = pd.read_csv(metadata_file, low_memory=False)

# Group by 'RGIId' and get the unique 'Form' value for each 'RGIId'
unique_forms = glathida_rgis.groupby('RGIId')['Form'].first().reset_index()
unique_term  = glathida_rgis.groupby('RGIId')['TermType'].first().reset_index()
# Function to convert 'Form' value to 2D vector
def convert_to_2d_vector(form_value):
    return [1, 0] if form_value == 1 else [0, 1]

def convert_to_4d_vector(term_value):
    if term_value == 0:
        return [1, 0, 0, 0]
    elif term_value == 1:
        return [0, 1, 0, 0]
    elif term_value == 2:
        return [0, 0, 1, 0]
    elif term_value == 5:
        return [0, 0, 0, 1]
    
# Apply the conversion function
unique_forms['Form_Vector'] = unique_forms['Form'].apply(convert_to_2d_vector)
form_vectors_array = np.array(unique_forms['Form_Vector'].tolist())

# Apply the conversion function
unique_term['Term_Vector'] = unique_term['TermType'].apply(convert_to_4d_vector)
term_vectors_array = np.array(unique_term['Term_Vector'].tolist())


# Display the resulting DataFrame
# print(unique_forms[['RGIId', 'Form_Vector']])
# unique_forms['Form_Vector']
# unique_term['Term_Vector']


In [6]:
def vectorize_image(image_path, threshold=128):
    """
    Reads an image and vectorizes it to 0 and 1 depending on the color.

    Parameters:
    - image_path: str, path to the image file

    Returns:
    - binary_vector: numpy array of 0s and 1s
    """
    # Open the image and convert to RGB
    image = Image.open(image_path).convert('RGB')
    
    # Convert the image to a numpy array
    image_array = np.array(image)
    
    # Define a blue color range
    lower_blue = np.array([0, 0, 128])
    upper_blue = np.array([127, 127, 255])
    
    # Create a mask for blue pixels
    blue_mask = np.all((image_array >= lower_blue) & (image_array <= upper_blue), axis=-1)
    
    # Create a binary vector where blue is 1 and everything else is 0
    binary_vector = blue_mask.astype(int)
    
    return binary_vector

In [8]:
def process_folder(folder_path):
    """
    Processes all images in the specified folder, vectorizing each one.
    
    Parameters:
    - folder_path: str, path to the folder containing images

    Returns:
    - result_dict: dictionary, keys are image filenames and values are their binary vectors
    """
    result_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            binary_image = vectorize_image(image_path)
            result_dict[filename] = binary_image
    
    return result_dict

In [10]:
# Example usage
folder_path = r'C:\Users\damsg\OGGM\glacier_geometries_images_small_rotmir'  
processed_images = process_folder(folder_path)

def resize_array(array, size=(64, 64)):
    # Convert the array to PIL Image
    image = Image.fromarray(np.array(array, dtype='uint8'))
    # Resize the image
    resized_image = image.resize(size, resample=Image.BILINEAR)
    # Convert the resized image back to numpy array
    return np.array(resized_image)

# Assuming 'data_dict' is your dictionary containing multiple arrays
resized_arrays = [resize_array(array) for array in processed_images.values()]

# Stack all resized arrays into a single numpy array
resized_data_array = np.stack(resized_arrays, axis=0)


## CNN

In [203]:
img_rows, img_cols = 64, 64
num_classes = 4 #Change to 2 for form to 4 for Termtype
model = Sequential()

model.add(layers.Conv2D(filters=32,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))

model.add(layers.MaxPooling2D(pool_size=2, strides=None))

model.add(layers.Conv2D(filters=64,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu'))

model.add(layers.MaxPooling2D(pool_size=2, strides=None))

model.add(layers.Conv2D(filters=128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))

model.add(layers.MaxPooling2D(pool_size=2, strides=None))

model.add(layers.Conv2D(filters=256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu'))

model.add(layers.MaxPooling2D(pool_size=2, strides=None))

model.add(layers.Dropout(rate=0.60))

model.add(layers.Flatten())

model.add(Dense(units=64, activation='relu'))

model.add(Dense(units=num_classes, activation='softmax'))

model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_64 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d_64 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_65 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_65 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_66 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_66 (MaxPoolin  (None, 8, 8, 128)      

In [205]:
print(len(resized_data_array))
print(len(form_vectors_array))
y = [form_vectors_array[i] for i in range(len(form_vectors_array)) for _ in range(8)]
y1 = [term_vectors_array[i] for i in range(len(term_vectors_array)) for _ in range(8)]

print(len(y))
print(y[0])
X = resized_data_array

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size = 0.8, random_state=1)
X_train0, X_test0, y_train1, y_test1 = train_test_split(X, y1,train_size = 0.8, random_state=1)


18568
2321
18568
[0 1]


In [207]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['categorical_accuracy'])
print('--------- TRAINING ---------')
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(x = np.array(X_train0), y = np.array(y_train1),validation_data=(np.array(X_test0), np.array(y_test1)), 
                    epochs = 1, callbacks=[early_stopping]) 

--------- TRAINING ---------
465/465 [==============================] - 67s 142ms/step - loss: 1.1678 - categorical_accuracy: 0.6505 - val_loss: 0.8387 - val_categorical_accuracy: 0.8220


## Normal NN

In [209]:
# X0 = glathida_rgis.drop(['ith_m','ith_f','THICKNESS','RGIId'],axis=1) 
X0 = glathida_rgis.drop(['Form','RGIId','ith_m','ith_f','THICKNESS'],axis=1) 
X0 = glathida_rgis.drop(['TermType','RGIId','ith_m','ith_f','THICKNESS'],axis=1) 

scaler = MinMaxScaler()

X0 = pd.DataFrame(scaler.fit_transform(X0.values), columns=X0.columns, index=X0.index)
# y0 = glathida_rgis['Form'] 
y0 = glathida_rgis['TermType'] 

X_train0, X_test0, y_train0, y_test0 = train_test_split(X0, y0,train_size = 0.8, random_state=1)
print(np.shape(X0))
model0 = Sequential([
    Dense(54,activation='LeakyReLU'   ,name='input_layer'),
    Dense(90,activation='relu'   ,name='hidden_layer1'),
    Dense(90,activation='relu'   ,name='hidden_layer2'),
    Dense(1, name='output')])

model0.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['categorical_accuracy'])

print('--------- TRAINING ---------')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model0.fit(x = np.array(X_train0), y = np.array(y_train0),validation_data=(np.array(X_test0), np.array(y_test0)), 
                    epochs = 5, callbacks=[early_stopping]) 


(81290, 53)
--------- TRAINING ---------
Epoch 1/5
2033/2033 [==============================] - 6s 3ms/step - loss: 5.2049e-08 - categorical_accuracy: 1.0000 - val_loss: 5.0307e-08 - val_categorical_accuracy: 1.0000
Epoch 2/5
2033/2033 [==============================] - 5s 3ms/step - loss: 5.2049e-08 - categorical_accuracy: 1.0000 - val_loss: 5.0307e-08 - val_categorical_accuracy: 1.0000
Epoch 3/5
2033/2033 [==============================] - 6s 3ms/step - loss: 5.2049e-08 - categorical_accuracy: 1.0000 - val_loss: 5.0307e-08 - val_categorical_accuracy: 1.0000
Epoch 4/5
2033/2033 [==============================] - 7s 3ms/step - loss: 5.2049e-08 - categorical_accuracy: 1.0000 - val_loss: 5.0307e-08 - val_categorical_accuracy: 1.0000
Epoch 5/5
2033/2033 [==============================] - 7s 4ms/step - loss: 5.2049e-08 - categorical_accuracy: 1.0000 - val_loss: 5.0307e-08 - val_categorical_accuracy: 1.0000


In [213]:
# perm_importance = permutation_importance(model0, X_test0, y_test0, scoring ='neg_mean_absolute_error', n_repeats=5, random_state=42)


In [215]:
# perm_importance = permutation_importance(model0, X_test0, y_test0, scoring ='neg_mean_absolute_error', n_repeats=5, random_state=42)

feature_importance_scores = perm_importance.importances_mean

# Get feature names
feature_names = X0.columns  # Assuming X is a pandas DataFrame with feature names

# Print or visualize feature importance
sorted_feature_indices = feature_importance_scores.argsort()[::-1]
sorted_feature_names = feature_names[sorted_feature_indices]

for i, feature_name in enumerate(sorted_feature_names):
    print(f"{i+1}. {feature_name}: {feature_importance_scores[sorted_feature_indices[i]]}")


1. Zmed: 0.005709929073745945
2. Aspect: 0.004124656653926662
3. Form: 0.0032511009947928217
4. POINT_LAT: 0.0028006188892364613
5. RGI: 0.0015365256226289703
6. aspect_gfa: 0.0013626384053841202
7. aspect_300: 0.0010358497998848803
8. Zmax: 0.00052399357424735
9. smb: 0.0003666735084903561
10. aspect_50: 0.00034876667908219084
11. slope_lon_gf100: 0.00027349420798070676
12. curv_300: 0.0002556475910718836
13. slope_lon_gf300: 0.00022655289792580246
14. dist_from_border_km_geom: 0.00018556618321193818
15. slope_lon_gf450: 0.00015092586798735042
16. slope_lat: 0.00010904609561013245
17. vy_gf150: 0.00010066974887859014
18. vx_gf150: 5.431998820853989e-05
19. vx_gf300: 2.8750925930309813e-05
20. curv_50: 2.5984332648920637e-05
21. vy: 1.5212649709073211e-05
22. vy_gf300: 1.2787207047448845e-05
23. vy_gf100: 1.0764569195886864e-05
24. vx_gfa: 3.74171580541649e-06
25. vy_gf50: -1.527155426273019e-06
26. dvx_dy: -3.333697056728813e-06
27. dvy_dy: -8.650500142592943e-06
28. dvy_dx: -1.224542

# Combine CNN with NN

## Form CNN + NN

In [227]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, concatenate
num_features = 53 #Features in dataset
# Define input layers for both image and non-image data
image_input = Input(shape=(64, 64, 1))  # Assuming 64x64 RGB images
non_image_input = Input(shape=(num_features,))  # Specify the number of features for your non-image data

# CNN for image data
conv1 = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = layers.Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = layers.Conv2D(256, (3, 3), activation='relu')(pool3)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
flatten = layers.Flatten()(pool4)

# Add the dense layer with 64 units and ReLU activation
dense_64_relu = Dense(64, activation='relu')(flatten)

# Concatenate the dense layer output with non-image data
combined_input = concatenate([dense_64_relu, non_image_input])

# Fully connected layers for combined data
fc1 = Dense(117, activation='relu')(combined_input)
fc2 = Dense(180, activation='relu')(fc1)
fc3 = Dense(180, activation='relu')(fc2)
fc3 = Dense(32, activation='relu')(fc2)
output = Dense(1, activation='sigmoid')(fc3)  # Output layer for binary classification

# Define the combined model
combined_model = Model(inputs=[image_input, non_image_input], outputs=output)

# Compile the model
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
combined_model.summary()


# history = combined_model.fit(inputs = [resized_data_array,]) 

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d_72 (Conv2D)             (None, 62, 62, 32)   320         ['input_23[0][0]']               
                                                                                                  
 max_pooling2d_72 (MaxPooling2D  (None, 31, 31, 32)  0           ['conv2d_72[0][0]']              
 )                                                                                                
                                                                                                  
 conv2d_73 (Conv2D)             (None, 29, 29, 64)   18496       ['max_pooling2d_72[0][0]']

In [12]:
# Example usage
folder_path = r'C:\Users\damsg\OGGM\glacier_geometries_images_small'  
processed_images = process_folder(folder_path)

def resize_array(array, size=(64, 64)):
    # Convert the array to PIL Image
    image = Image.fromarray(np.array(array, dtype='uint8'))
    # Resize the image
    resized_image = image.resize(size, resample=Image.BILINEAR)
    # Convert the resized image back to numpy array
    return np.array(resized_image)

# Assuming 'data_dict' is your dictionary containing multiple arrays
resized_arrays = [resize_array(array) for array in processed_images.values()]

# Stack all resized arrays into a single numpy array
resized_data_array = np.stack(resized_arrays, axis=0)

# Apply the conversion function
unique_forms = glathida_rgis.groupby('RGIId')['Form'].first().reset_index()
unique_forms['Form_Vector'] = unique_forms['Form'].apply(convert_to_2d_vector)
form_vectors_array = np.array(unique_forms['Form_Vector'].tolist())

# Apply the conversion function
unique_term  = glathida_rgis.groupby('RGIId')['TermType'].first().reset_index()
unique_term['Term_Vector'] = unique_term['TermType'].apply(convert_to_4d_vector)
term_vectors_array = np.array(unique_term['Term_Vector'].tolist())

In [13]:
pictures = []
current_rgi_id = None
i = -1  # Initial value of i

for rgi_id in glathida_rgis['RGIId']:
    if rgi_id != current_rgi_id:
        i += 1  # Increment i when RGIId changes
        current_rgi_id = rgi_id
    pictures.append(resized_data_array[i])

# Convert pictures to a numpy array if needed
pictures = np.array(pictures)
print(np.shape(pictures))
print(np.shape(glathida_rgis['RGIId']))

(81290, 64, 64)
(81290,)


In [16]:
# pictures = resized_data_array

y01 = glathida_rgis['Form'] 
y02_values = glathida_rgis['TermType']

y02 = np.array([convert_to_4d_vector(term_value) for term_value in y02_values])
# y02_one_hot = [convert_to_4d_vector(term_value) for term_value in y02_values]

# Convert the list to a numpy array if needed
# y02 = np.array(y02_one_hot)


y03 = glathida_rgis['THICKNESS']

X01 = glathida_rgis.drop(['Form','RGIId','ith_m','ith_f','THICKNESS'],axis=1) 
X02 = glathida_rgis.drop(['TermType','RGIId','ith_m','ith_f','THICKNESS'],axis=1) 
X03 = glathida_rgis.drop(['RGIId','ith_m','ith_f','THICKNESS'],axis=1) 

scaler = MinMaxScaler()

X01 = pd.DataFrame(scaler.fit_transform(X01.values), columns=X01.columns, index=X01.index)
X02 = pd.DataFrame(scaler.fit_transform(X02.values), columns=X02.columns, index=X02.index)
X03 = pd.DataFrame(scaler.fit_transform(X03.values), columns=X03.columns, index=X03.index)

unique_forms = glathida_rgis.groupby('RGIId')['Form'].first().reset_index()
unique_term  = glathida_rgis.groupby('RGIId')['TermType'].first().reset_index()

X01_train, X01_test, y01_train, y01_test = train_test_split(X01, y01,train_size = 0.8, random_state=1)
X02_train, X02_test, y02_train, y02_test = train_test_split(X02, y02,train_size = 0.8, random_state=1)
X03_train, X03_test, y03_train, y03_test = train_test_split(X03, y03,train_size = 0.8, random_state=1)
#Shooting for form
pictures_train, pictures_val, X01_train, X01_val, y01_train, y01_val = train_test_split(
    pictures, X01, y01, test_size=0.2, random_state=42)

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=2,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights of the best epoch
)
checkpoint = ModelCheckpoint(
    r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/Form_model_weights.h5',  # Full file path to save the model weights
    monitor='val_loss',               # Metric to monitor
    save_best_only=True,              # Save only the best weights
    save_weights_only=True            # Save only the weights (not the entire model)
)

combined_model.fit(
    [pictures_train, X01_train],  # Inputs: list of image data and non-image data
    y01_train,                    # Target labels
    epochs=7,                    # Number of epochs
    batch_size=32,                # Batch size
    validation_data=([pictures_val, X01_val], y01_val),  # Validation data
    callbacks=[early_stopping,checkpoint]    # Include the early stopping callback
)
combined_model.save_weights(r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/Form_model_weights_final.h5')

## Termtype CNN + NN

In [232]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, concatenate
num_features = 53 #Features in dataset
# Define input layers for both image and non-image data
image_input = Input(shape=(64, 64, 1))  # Assuming 64x64 Greyscale images
non_image_input = Input(shape=(num_features,))  # Specify the number of features for your non-image data

# CNN for image data
conv1 = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = layers.Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = layers.Conv2D(256, (3, 3), activation='relu')(pool3)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
flatten = layers.Flatten()(pool4)

# Add the dense layer with 64 units and ReLU activation
dense_64_relu = Dense(64, activation='relu')(flatten)

# Concatenate the dense layer output with non-image data
combined_input = concatenate([dense_64_relu, non_image_input])

# Fully connected layers for combined data
fc1 = Dense(117, activation='relu')(combined_input)
fc2 = Dense(180, activation='relu')(fc1)
fc3 = Dense(180, activation='relu')(fc2)
fc3 = Dense(32, activation='relu')(fc2)
output = Dense(4, activation='sigmoid')(fc3)  # Output layer for multi classification

# Define the combined model
combined_model = Model(inputs=[image_input, non_image_input], outputs=output)

# Compile the model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
combined_model.summary()


# history = combined_model.fit(inputs = [resized_data_array,]) 

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d_80 (Conv2D)             (None, 62, 62, 32)   320         ['input_27[0][0]']               
                                                                                                  
 max_pooling2d_80 (MaxPooling2D  (None, 31, 31, 32)  0           ['conv2d_80[0][0]']              
 )                                                                                                
                                                                                                  
 conv2d_81 (Conv2D)             (None, 29, 29, 64)   18496       ['max_pooling2d_80[0][0]']

In [234]:
#Shooting for TermType
pictures_train, pictures_val, X02_train, X02_val, y02_train, y02_val = train_test_split(
    pictures, X02, y02, test_size=0.2, random_state=42)

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights of the best epoch
)

checkpoint = ModelCheckpoint(
    r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/Term_model_weights.h5',  # Full file path to save the model weights
    monitor='val_loss',               # Metric to monitor
    save_best_only=True,              # Save only the best weights
    save_weights_only=True            # Save only the weights (not the entire model)
)
# Train the model with early stopping
combined_model.fit(
    [pictures_train, X02_train],  # Inputs: list of image data and non-image data
    y02_train,                    # Target labels
    epochs=7,                    # Number of epochs
    batch_size=32,                # Batch size
    validation_data=([pictures_val, X02_val], y02_val),  # Validation data
    callbacks=[early_stopping,checkpoint]    # Include the early stopping callback
)

combined_model.save_weights(r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/Term_model_weights_final.h5')

Epoch 1/7
2033/2033 [==============================] - 317s 155ms/step - loss: 0.1012 - accuracy: 0.9629 - val_loss: 0.0768 - val_accuracy: 0.9705
Epoch 2/7
2033/2033 [==============================] - 317s 156ms/step - loss: 0.0107 - accuracy: 0.9966 - val_loss: 0.0455 - val_accuracy: 0.9814
Epoch 3/7
2033/2033 [==============================] - 322s 158ms/step - loss: 0.0047 - accuracy: 0.9987 - val_loss: 0.0011 - val_accuracy: 0.9997
Epoch 4/7
2033/2033 [==============================] - 324s 159ms/step - loss: 0.0057 - accuracy: 0.9982 - val_loss: 1.8127e-04 - val_accuracy: 0.9999
Epoch 5/7
2033/2033 [==============================] - 322s 158ms/step - loss: 0.0053 - accuracy: 0.9982 - val_loss: 0.0138 - val_accuracy: 0.9945
Epoch 6/7
2033/2033 [==============================] - 317s 156ms/step - loss: 2.5115e-04 - accuracy: 0.9999 - val_loss: 2.7377e-06 - val_accuracy: 1.0000
Epoch 7/7
2033/2033 [==============================] - 322s 158ms/step - loss: 1.5606e-06 - accuracy: 1.00

## Thickness CNN + NN

In [29]:
num_features = 54 #Features in dataset
# Define input layers for both image and non-image data
image_input = Input(shape=(64, 64, 1))  # Assuming 64x64 RGB images
non_image_input = Input(shape=(num_features,))  # Specify the number of features for your non-image data

# CNN for image data
conv1 = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = layers.Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = layers.Conv2D(256, (3, 3), activation='relu')(pool3)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
flatten = layers.Flatten()(pool4)

# Add the dense layer with 64 units and ReLU activation
dense_64_relu = Dense(64, activation='relu')(flatten)

# Concatenate the dense layer output with non-image data
combined_input = concatenate([dense_64_relu, non_image_input])

# Fully connected layers for combined data
fc1 = Dense(117, activation='relu')(combined_input)
fc2 = Dense(150, activation='relu')(fc1)
fc3 = Dense(75, activation='relu')(fc2)
fc3 = Dense(32, activation='relu')(fc2)
output = Dense(1)(fc3)  # Output layer for regression

# Define the combined model
combined_model = Model(inputs=[image_input, non_image_input], outputs=output)

# Compile the model
combined_model.compile(optimizer='adam',loss= 'mean_absolute_error',metrics=['mean_absolute_error'])
# Print the model summary
# combined_model.summary()

In [31]:
#Shooting for TermType
pictures_train, pictures_val, X03_train, X03_val, y03_train, y03_val = train_test_split(
    pictures, X03, y03, test_size=0.2, random_state=42)

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights of the best epoch
)
checkpoint = ModelCheckpoint(
    r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/Thick_model_weights.h5',  # Full file path to save the model weights
    monitor='val_loss',               # Metric to monitor
    save_best_only=True,              # Save only the best weights
    save_weights_only=True            # Save only the weights (not the entire model)
)

# Train the model with early stopping
combined_model.fit(
    [pictures_train, X03_train],  # Inputs: list of image data and non-image data
    y03_train,                    # Target labels
    epochs=100,                   # Number of epochs
    batch_size=32,                # Batch size
    validation_data=([pictures_val, X03_val], y03_val),  # Validation data
    callbacks=[early_stopping,checkpoint]    # Include the early stopping callback
)

combined_model.save_weights(r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/Thick_model_weights_final.h5')


Epoch 1/100
2033/2033 [==============================] - 239s 117ms/step - loss: 78.2529 - mean_absolute_error: 78.2529 - val_loss: 57.1098 - val_mean_absolute_error: 57.1098
Epoch 2/100
2033/2033 [==============================] - 227s 112ms/step - loss: 52.1564 - mean_absolute_error: 52.1564 - val_loss: 48.8451 - val_mean_absolute_error: 48.8451
Epoch 3/100
2033/2033 [==============================] - 277s 136ms/step - loss: 48.3044 - mean_absolute_error: 48.3044 - val_loss: 47.0176 - val_mean_absolute_error: 47.0176
Epoch 4/100
2033/2033 [==============================] - 230s 113ms/step - loss: 46.4882 - mean_absolute_error: 46.4882 - val_loss: 46.1673 - val_mean_absolute_error: 46.1673
Epoch 5/100
2033/2033 [==============================] - 237s 117ms/step - loss: 45.4935 - mean_absolute_error: 45.4935 - val_loss: 48.8119 - val_mean_absolute_error: 48.8119
Epoch 6/100
2033/2033 [==============================] - 242s 119ms/step - loss: 44.7931 - mean_absolute_error: 44.7931 - val

In [ ]:
#combined_model.load_weights('r'C:/Users/damsg/Desktop/Python/AppliedMachineLearning/Final_Project_Glacier/model_weights_final.h5'')
